In [4]:
##PART 1.  Preparing the dataset

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Text Preprocessing
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import texthero as hero
import snowballstemmer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import metrics


In [6]:
##read the dataset
import pandas as pd
df = pd.read_csv('netflix_titles.csv')   

print(df.shape)  ### There are 8807 TV/movies in the dataset
df.head()

(8807, 12)


show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021          2021  TV-MA   1 Season   
3  September 24, 2021          2021  TV-MA   1 Season   
4  September 24, 2021          2021  TV-MA  2 Seasons   

                                           listed_in  \
0                                      Documentaries   
1    International TV Shows, TV Dramas, TV Mysteries   
2  Crime TV Shows, International TV Shows, TV Act...   
3                             Docuseries, Reality TV   
4  International TV Shows, Romantic TV Shows, TV ...   

                                         description  
0  As her father nears the end of his life, filmm...  
1  After crossing paths at a party, a Cape Town t...  
2  To protect his family from a powerful drug lor...  
3  Feuds, flirtations and toilet talk go down amo...  
4  In a city of coaching centers known to train I...

In [7]:
df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [8]:
#take the first genre from "listed_in" as genre
df['Genre'] = df['listed_in'].apply(lambda x: x.split(',')[0])
df['Genre']

0                  Documentaries
1         International TV Shows
2                 Crime TV Shows
3                     Docuseries
4         International TV Shows
                  ...           
8802                 Cult Movies
8803                    Kids' TV
8804                    Comedies
8805    Children & Family Movies
8806                      Dramas
Name: Genre, Length: 8807, dtype: object

In [9]:
#check numbers of TV/movies in ecah genre 
df['Genre'].value_counts()

Dramas                          1600
Comedies                        1210
Action & Adventure               859
Documentaries                    829
International TV Shows           774
Children & Family Movies         605
Crime TV Shows                   399
Kids' TV                         388
Stand-Up Comedy                  334
Horror Movies                    275
British TV Shows                 253
Docuseries                       221
Anime Series                     176
International Movies             128
Reality TV                       120
TV Comedies                      120
Classic Movies                    80
TV Dramas                         67
Thrillers                         65
Movies                            57
TV Action & Adventure             40
Stand-Up Comedy & Talk Shows      34
Romantic TV Shows                 32
Classic & Cult TV                 22
Anime Features                    21
Independent Movies                20
Music & Musicals                  18
T

In [10]:
##pick top 6 genres

genre = ['Dramas','Comedies','Action & Adventure','Documentaries','International TV Shows', 'Children & Family Movies']
len(df[df.Genre.isin(genre)])/len(df)

0.6673100942432156

In [11]:
# Filter out top 6 genres
df = df[df['Genre'].isin(genre)]
df.shape

(5877, 13)

In [12]:
data = df[['Genre','description']]
data

Genre  \
0                Documentaries   
1       International TV Shows   
4       International TV Shows   
6     Children & Family Movies   
7                       Dramas   
...                        ...   
8800    International TV Shows   
8801                    Dramas   
8804                  Comedies   
8805  Children & Family Movies   
8806                    Dramas   

                                            description  
0     As her father nears the end of his life, filmm...  
1     After crossing paths at a party, a Cape Town t...  
4     In a city of coaching centers known to train I...  
6     Equestria's divided. But a bright-eyed hero be...  
7     On a photo shoot in Ghana, an American model s...  
...                                                 ...  
8800  Strong-willed, middle-class Kashaf and carefre...  
8801  Recovering alcoholic Talal wakes up inside a s...  
8804  Looking to survive in a world taken over by zo...  
8805  Dragged from civilian life, a former superhero...  
8806  A scrappy but poor boy worms his way into a ty...  

[5877 rows x 2 columns]

In [13]:
#Lemmatization
def lemma(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    t=[lemmatizer.lemmatize(word) for word in tokens]
    text=' '.join(t)
    return text

#Stemming
def stem(text):
    stemmer = snowballstemmer.stemmer('english')
    tokens = word_tokenize(text)
    t=[stemmer.stemWord(word) for word in tokens]
    text=' '.join(t)
    return text

In [14]:
# clean the news content by using clean method from hero package
data.description = hero.clean(data.description)

data.description = data.description.apply(lemma)

data.description = data.description.apply(stem)

D:\anaconda\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [15]:
#Define X and y for text data

X = data['description']  #input
y = data['Genre'] #output
print(X.shape)
print(y.shape)

(5877,)
(5877,)


In [16]:
from collections import Counter
Counter(y)

Counter({'Documentaries': 829,
         'International TV Shows': 774,
         'Children & Family Movies': 605,
         'Dramas': 1600,
         'Comedies': 1210,
         'Action & Adventure': 859})

In [17]:
category_df = pd.get_dummies(data.Genre)
category_df['description'] = data.description
category_df.head()

Action & Adventure  Children & Family Movies  Comedies  Documentaries  \
0                   0                         0         0              1   
1                   0                         0         0              0   
4                   0                         0         0              0   
6                   0                         1         0              0   
7                   0                         0         0              0   

   Dramas  International TV Shows  \
0       0                       0   
1       0                       1   
4       0                       1   
6       0                       0   
7       1                       0   

                                         description  
0  father near end life filmmak kirsten johnson s...  
1  cross path parti cape town teen set prove whet...  
4  citi coach center known train india ' finest c...  
6  equestria divid bright eye hero belief earth p...  
7  photo shoot ghana american model slip back tim...

In [18]:
#Define X and y for text data

X = category_df['description']  #input
y = category_df[category_df.columns[ :6]] #output
print(X.shape)
print(y.shape)

(5877,)
(5877, 6)


In [19]:
y

Action & Adventure  Children & Family Movies  Comedies  Documentaries  \
0                      0                         0         0              1   
1                      0                         0         0              0   
4                      0                         0         0              0   
6                      0                         1         0              0   
7                      0                         0         0              0   
...                  ...                       ...       ...            ...   
8800                   0                         0         0              0   
8801                   0                         0         0              0   
8804                   0                         0         1              0   
8805                   0                         1         0              0   
8806                   0                         0         0              0   

      Dramas  International TV Shows  
0          0                       0  
1          0                       1  
4          0                       1  
6          0                       0  
7          1                       0  
...      ...                     ...  
8800       0                       1  
8801       1                       0  
8804       0                       0  
8805       0                       0  
8806       1                       0  

[5877 rows x 6 columns]

In [20]:
# split X and y into 75%training and 25%testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)
print(X_train.shape)
print(X_test.shape)

(4701,)
(1176,)


In [21]:
X_train

2067    discov human face extinct new recruit chaldea ...
7699    documentari centuri old hors race citi siena i...
4985    young woman amnesia becom key figur miss perso...
3876    world known copper dimens group high school gi...
7362    three men differ nation must find way amazon j...
                              ...                        
1351    dramat virgin mari work miracl girl mexico fou...
7890    save parent ' home lotteri squander year old m...
6234    barbi sister set swiss adventur majest alp exc...
355     street wise defens lawyer oper back seat linco...
7848    five acclaim photograph travel world provid de...
Name: description, Length: 4701, dtype: object

In [22]:
vect = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')

vect.fit(X_train)

X_train_dtm = vect.transform(X_train)
X_test_dtm = vect.transform(X_test)

In [23]:
X_train_dtm

<4701x121897 sparse matrix of type '<class 'numpy.float64'>'
	with 190391 stored elements in Compressed Sparse Row format>

In [24]:
###Part 2: LogisticRegression

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier


LR_pipeline = Pipeline([('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1))])
accuracy = 0
for category in category_df.columns[ :6]:
    print('**Processing {} titles...**'.format(category))
    
    # Training logistic regression model on train data
    LR_pipeline.fit(X_train_dtm, y_train[category])
    
    # calculating test accuracy
    prediction = LR_pipeline.predict(X_test_dtm)
    accuracy = accuracy + metrics.accuracy_score(y_test[category], prediction)
    print("Accuracy:{}".format(metrics.accuracy_score(y_test[category], prediction)))
    print("F1 score:{}".format(metrics.f1_score(y_test[category], prediction, average = 'macro')))
    print('AUC ROC score is {}'.format(metrics.roc_auc_score(y_test[category],prediction)))
    print("\n")

print('Test averaged Accuracy is {}'.format(accuracy/len(category_df.columns[ :6])))

**Processing Action & Adventure titles...**
Accuracy:0.8460884353741497
F1 score:0.4742413950673698
AUC ROC score is 0.5076931966387664


**Processing Children & Family Movies titles...**
Accuracy:0.9039115646258503
F1 score:0.4834386114944316
AUC ROC score is 0.5043859649122807


**Processing Comedies titles...**
Accuracy:0.7848639455782312
F1 score:0.4584561172318518
AUC ROC score is 0.5086786684782608


**Processing Documentaries titles...**
Accuracy:0.9039115646258503
F1 score:0.7131676361713379
AUC ROC score is 0.6590145372196655


**Processing Dramas titles...**
Accuracy:0.7474489795918368
F1 score:0.4931380217592263
AUC ROC score is 0.5299794214773805


**Processing International TV Shows titles...**
Accuracy:0.8690476190476191
F1 score:0.46496815286624205
AUC ROC score is 0.5


Test averaged Accuracy is 0.8425453514739228


In [26]:
###Part 3: SVM

In [27]:
from sklearn.svm import LinearSVC

SVC_pipeline = Pipeline([('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1))])

accuracy = 0
for category in category_df.columns[ :6]:
    print('**Processing {} titles...**'.format(category))
    
    # Training logistic regression model on train data
    SVC_pipeline.fit(X_train_dtm, y_train[category])
    
    # calculating test accuracy
    prediction = SVC_pipeline.predict(X_test_dtm)
    accuracy = accuracy + metrics.accuracy_score(y_test[category], prediction)
    print("Accuracy:{}".format(metrics.accuracy_score(y_test[category], prediction)))
    print("F1 score:{}".format(metrics.f1_score(y_test[category], prediction, average = 'macro')))
    print('AUC ROC is {}'.format(metrics.roc_auc_score(y_test[category],prediction)))
    print("\n")

print('Test averaged Accuracy is {}'.format(accuracy/len(category_df.columns[ :6])))

**Processing Action & Adventure titles...**
Accuracy:0.8818027210884354
F1 score:0.7156034970205734
AUC ROC is 0.6759364733462102


**Processing Children & Family Movies titles...**
Accuracy:0.9345238095238095
F1 score:0.7490626428898035
AUC ROC is 0.689686787590445


**Processing Comedies titles...**
Accuracy:0.8231292517006803
F1 score:0.677670660292327
AUC ROC is 0.6515455163043479


**Processing Documentaries titles...**
Accuracy:0.9336734693877551
F1 score:0.8346873896149771
AUC ROC is 0.7851924903206954


**Processing Dramas titles...**
Accuracy:0.7585034013605442
F1 score:0.6621619434508877
AUC ROC is 0.652267948107638


**Processing International TV Shows titles...**
Accuracy:0.8818027210884354
F1 score:0.5881761633751293
AUC ROC is 0.5680039138943248


Test averaged Accuracy is 0.86890589569161


In [28]:
from sklearn.svm import SVC

SVC_pipeline = Pipeline([('clf', OneVsRestClassifier(SVC(kernel='rbf', C=10, gamma=0.1 ), n_jobs=1))])

accuracy = 0
for category in category_df.columns[ :6]:
    print('**Processing {} titles...**'.format(category))
    
    # Training logistic regression model on train data
    SVC_pipeline.fit(X_train_dtm, y_train[category])
    
    # calculating test accuracy
    prediction = SVC_pipeline.predict(X_test_dtm)
    accuracy = accuracy + metrics.accuracy_score(y_test[category], prediction)
    print("Accuracy:{}".format(metrics.accuracy_score(y_test[category], prediction)))
    print("F1 score:{}".format(metrics.f1_score(y_test[category], prediction, average = 'macro')))
    print('AUC ROC is {}'.format(metrics.roc_auc_score(y_test[category],prediction)))
    print("\n")

print('Test averaged Accuracy is {}'.format(accuracy/len(category_df.columns[ :6])))

**Processing Action & Adventure titles...**
Accuracy:0.8818027210884354
F1 score:0.7352170807206921
AUC ROC is 0.7026810625196045


**Processing Children & Family Movies titles...**
Accuracy:0.9379251700680272
F1 score:0.7792923307118599
AUC ROC is 0.7268064228367529


**Processing Comedies titles...**
Accuracy:0.8197278911564626
F1 score:0.6950207468879668
AUC ROC is 0.673335597826087


**Processing Documentaries titles...**
Accuracy:0.935374149659864
F1 score:0.8420248868778281
AUC ROC is 0.7965519760391555


**Processing Dramas titles...**
Accuracy:0.7525510204081632
F1 score:0.6674127339397113
AUC ROC is 0.6619331066822098


**Processing International TV Shows titles...**
Accuracy:0.8894557823129252
F1 score:0.6658594371541225
AUC ROC is 0.6275573741327166


Test averaged Accuracy is 0.8694727891156463


In [29]:
###Part 4: MultinomialNB 多项式贝叶斯

In [30]:
from sklearn.naive_bayes import MultinomialNB

NB_pipeline = Pipeline([('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None)))])
accuracy = 0
for category in category_df.columns[ :6]:
    print('**Processing {} titles...**'.format(category))
    
    # Training logistic regression model on train data
    NB_pipeline.fit(X_train_dtm, y_train[category])
    
    # calculating test accuracy
    prediction = NB_pipeline.predict(X_test_dtm)
    accuracy = accuracy + metrics.accuracy_score(y_test[category], prediction)
    print("Accuracy:{}".format(metrics.accuracy_score(y_test[category], prediction)))
    print("F1 score:{}".format(metrics.f1_score(y_test[category], prediction, average = 'macro')))
    print('AUC ROC is {}'.format(roc_auc_score(y_test[category],prediction)))
    print("\n")

print('Test averaged Accuracy is {}'.format(accuracy/len(category_df.columns[ :6])))

**Processing Action & Adventure titles...**
Accuracy:0.8443877551020408
F1 score:0.45781466113416325
AUC ROC is 0.5


**Processing Children & Family Movies titles...**
Accuracy:0.9030612244897959
F1 score:0.47453083109919575
AUC ROC is 0.5


**Processing Comedies titles...**
Accuracy:0.7831632653061225
F1 score:0.44303186203950484
AUC ROC is 0.501953125


**Processing Documentaries titles...**
Accuracy:0.8630952380952381
F1 score:0.4693601863192146
AUC ROC is 0.5030864197530864


**Processing Dramas titles...**
Accuracy:0.7389455782312925
F1 score:0.42493887530562346
AUC ROC is 0.5


**Processing International TV Shows titles...**
Accuracy:0.8690476190476191
F1 score:0.46496815286624205
AUC ROC is 0.5


Test averaged Accuracy is 0.8336167800453514


In [31]:
###Part 5: RandomForest

In [32]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

RF_pipeline = Pipeline([('clf', OneVsRestClassifier(RandomForestClassifier(), n_jobs=1))])

accuracy = 0
for category in category_df.columns[ :6]:
    print('**Processing {} titles...**'.format(category))
    
    # Training logistic regression model on train data
    RF_pipeline.fit(X_train_dtm, y_train[category])
    
    # calculating test accuracy
    prediction = RF_pipeline.predict(X_test_dtm)
    accuracy = accuracy + metrics.accuracy_score(y_test[category], prediction)
    print("Accuracy:{}".format(metrics.accuracy_score(y_test[category], prediction)))
    print("F1 score:{}".format(metrics.f1_score(y_test[category], prediction, average = 'macro')))
    print('AUC ROC is {}'.format(roc_auc_score(y_test[category],prediction)))
    print("\n")

print('Test averaged Accuracy is {}'.format(accuracy/len(category_df.columns[ :6])))

**Processing Action & Adventure titles...**
Accuracy:0.8562925170068028
F1 score:0.5361293228800762
AUC ROC is 0.5404800818846681


**Processing Children & Family Movies titles...**
Accuracy:0.907312925170068
F1 score:0.5176101788677854
AUC ROC is 0.5219298245614035


**Processing Comedies titles...**
Accuracy:0.7874149659863946
F1 score:0.4836416262127397
AUC ROC is 0.5201766304347826


**Processing Documentaries titles...**
Accuracy:0.9141156462585034
F1 score:0.7556888887974709
AUC ROC is 0.6960515742567024


**Processing Dramas titles...**
Accuracy:0.7423469387755102
F1 score:0.5130314994526648
AUC ROC is 0.5370600825389924


**Processing International TV Shows titles...**
Accuracy:0.8707482993197279
F1 score:0.4782121339498389
AUC ROC is 0.5064935064935064


Test averaged Accuracy is 0.8463718820861678


In [47]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

RF_pipeline = Pipeline([('clf', OneVsRestClassifier(RandomForestClassifier(n_estimators = 250), n_jobs=1))])

accuracy = 0
for category in category_df.columns[ :6]:
    print('**Processing {} titles...**'.format(category))
    
    # Training logistic regression model on train data
    RF_pipeline.fit(X_train_dtm, y_train[category])
    
    # calculating test accuracy
    prediction = RF_pipeline.predict(X_test_dtm)
    accuracy = accuracy + metrics.accuracy_score(y_test[category], prediction)
    print("Accuracy:{}".format(metrics.accuracy_score(y_test[category], prediction)))
    print("F1 score:{}".format(metrics.f1_score(y_test[category], prediction, average = 'macro')))
    print('AUC ROC is {}'.format(roc_auc_score(y_test[category],prediction)))
    print("\n")

print('Test averaged Accuracy is {}'.format(accuracy/len(category_df.columns[ :6])))

**Processing Action & Adventure titles...**
Accuracy:0.8554421768707483
F1 score:0.531245603669002
AUC ROC is 0.5377478414475096


**Processing Children & Family Movies titles...**
Accuracy:0.9081632653061225
F1 score:0.5258064516129033
AUC ROC is 0.5263157894736842


**Processing Comedies titles...**
Accuracy:0.7874149659863946
F1 score:0.486889503225266
AUC ROC is 0.521586277173913


**Processing Documentaries titles...**
Accuracy:0.9141156462585034
F1 score:0.7556888887974709
AUC ROC is 0.6960515742567024


**Processing Dramas titles...**
Accuracy:0.7482993197278912
F1 score:0.5334909377462569
AUC ROC is 0.549514024506809


**Processing International TV Shows titles...**
Accuracy:0.8707482993197279
F1 score:0.4782121339498389
AUC ROC is 0.5064935064935064


Test averaged Accuracy is 0.8473639455782314


In [40]:
from sklearn.model_selection import GridSearchCV

param_test1 = {'n_estimators':[50,120,160,200,250]}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt' ,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',cv=5)
gsearch1.fit(X_train_dtm, y_train[category])
print( gsearch1.best_params_, gsearch1.cv_results_)


{'n_estimators': 250} {'mean_fit_time': array([0.17703948, 0.43969851, 0.57952924, 0.72456408, 0.9166049 ]), 'std_fit_time': array([0.00905725, 0.02018005, 0.02584147, 0.0112542 , 0.02917903]), 'mean_score_time': array([0.00880527, 0.01940508, 0.02620635, 0.03240767, 0.04060903]), 'std_score_time': array([0.00040166, 0.00049021, 0.00074869, 0.00119971, 0.0004899 ]), 'param_n_estimators': masked_array(data=[50, 120, 160, 200, 250],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_estimators': 50}, {'n_estimators': 120}, {'n_estimators': 160}, {'n_estimators': 200}, {'n_estimators': 250}], 'split0_test_score': array([0.54374284, 0.59986872, 0.62475816, 0.60486339, 0.60960635]), 'split1_test_score': array([0.57355412, 0.6133529 , 0.61245355, 0.60791726, 0.60842129]), 'split2_test_score': array([0.55027969, 0.63755633, 0.6457938 , 0.64765674, 0.65364089]), 'split3_test_score': array([0.58866026, 0.64705388, 0.65635377, 

In [35]:
from sklearn.model_selection import GridSearchCV
param_test2 = {'max_depth':[1,2,3,5,7,9,11,13]}#, 'min_samples_split':[100,120,150,180,200,300]}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators=50, min_samples_split=100,
                                  min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=10),
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(X_train_dtm, y_train[category])
print( gsearch2.best_params_, gsearch2.cv_results_)


{'max_depth': 7} {'mean_fit_time': array([0.12823143, 0.15963593, 0.18044028, 0.19864445, 0.20484567,
       0.20804639, 0.20864973, 0.20784645]), 'std_fit_time': array([0.00271642, 0.00320065, 0.00412905, 0.0078398 , 0.0104981 ,
       0.01054763, 0.01224725, 0.01237051]), 'mean_score_time': array([0.00940204, 0.00920186, 0.00900216, 0.00880189, 0.00900226,
       0.00880194, 0.00920196, 0.00920205]), 'std_score_time': array([4.90173941e-04, 3.99780302e-04, 1.78416128e-07, 4.00161772e-04,
       4.62310777e-07, 4.00066404e-04, 3.99732718e-04, 4.00042658e-04]), 'param_max_depth': masked_array(data=[1, 2, 3, 5, 7, 9, 11, 13],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'max_depth': 1}, {'max_depth': 2}, {'max_depth': 3}, {'max_depth': 5}, {'max_depth': 7}, {'max_depth': 9}, {'max_depth': 11}, {'max_depth': 13}], 'split0_test_score': array([0.52679946, 0.55212323, 0.5464376 , 0.54115174, 0.54374

C:\Users\weime\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_search.py:821: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


In [37]:
from sklearn.model_selection import GridSearchCV
param_test3 = {'min_samples_split':[80,100,120,140], 'min_samples_leaf':[10,20,30,40,50,100]}
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 50, max_depth=2,
                                  max_features='sqrt' ,oob_score=True, random_state=10),
   param_grid = param_test3, scoring='roc_auc',iid=False, cv=5)
gsearch3.fit(X_train_dtm, y_train[category])
print( gsearch3.best_params_, gsearch3.cv_results_)


{'min_samples_leaf': 10, 'min_samples_split': 80} {'mean_fit_time': array([0.17043843, 0.17224131, 0.16983829, 0.17563901, 0.15443478,
       0.1548346 , 0.15943589, 0.15923543, 0.15003424, 0.14863324,
       0.15023603, 0.16023579, 0.14843264, 0.15163364, 0.14703426,
       0.14423208, 0.13763099, 0.14263191, 0.14303231, 0.14543219,
       0.12642817, 0.12582836, 0.1260283 , 0.126828  ]), 'std_fit_time': array([0.0031375 , 0.0038687 , 0.00160064, 0.00382706, 0.00313792,
       0.00231618, 0.00338285, 0.00299387, 0.00275749, 0.00265352,
       0.00146811, 0.00412003, 0.00366645, 0.0039804 , 0.0038479 ,
       0.0021361 , 0.00320058, 0.00523987, 0.00963523, 0.00398095,
       0.00280017, 0.00160052, 0.00089532, 0.00365586]), 'mean_score_time': array([0.00880175, 0.00900207, 0.00900183, 0.00920229, 0.00860233,
       0.00900202, 0.00940185, 0.00960226, 0.00900149, 0.00900197,
       0.00920238, 0.00940204, 0.00880189, 0.00980239, 0.00920043,
       0.009202  , 0.00900183, 0.00900187, 0.0

C:\Users\weime\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_search.py:821: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


In [38]:
from sklearn.model_selection import GridSearchCV
param_test4 = {'max_features':[3,5,7,9,11]}
gsearch4 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 50, max_depth=2, min_samples_split=80,
                                  min_samples_leaf=10 ,oob_score=True, random_state=10),
   param_grid = param_test4, scoring='roc_auc',iid=False, cv=5)
gsearch4.fit(X_train_dtm, y_train[category])
print( gsearch4.best_params_, gsearch4.cv_results_)


{'max_features': 3} {'mean_fit_time': array([0.07641673, 0.0788208 , 0.0800209 , 0.0828187 , 0.07941766]), 'std_fit_time': array([0.00149723, 0.00075352, 0.00167173, 0.00285754, 0.00120029]), 'mean_score_time': array([0.00900235, 0.00879884, 0.00900211, 0.00900216, 0.00880179]), 'std_score_time': array([5.56082906e-07, 7.47652534e-04, 4.67203091e-07, 8.17605410e-07,
       4.00114074e-04]), 'param_max_features': masked_array(data=[3, 5, 7, 9, 11],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'max_features': 3}, {'max_features': 5}, {'max_features': 7}, {'max_features': 9}, {'max_features': 11}], 'split0_test_score': array([0.5       , 0.49265606, 0.49613555, 0.49388005, 0.49326805]), 'split1_test_score': array([0.5       , 0.49476202, 0.49476202, 0.48823925, 0.48270972]), 'split2_test_score': array([0.50183824, 0.50060286, 0.50060286, 0.50334045, 0.5017987 ]), 'split3_test_score': array([0.50061275, 0.50025202, 0.5

C:\Users\weime\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_search.py:821: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(


In [44]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

RF_pipeline = Pipeline([('clf', OneVsRestClassifier(RandomForestClassifier(n_estimators = 250, max_depth=7,min_samples_split=80,
                                  min_samples_leaf=10,max_features=3 ,oob_score=True, random_state=10), n_jobs=1))])

accuracy = 0
for category in category_df.columns[ :6]:
    print('**Processing {} titles...**'.format(category))
    
    # Training logistic regression model on train data
    RF_pipeline.fit(X_train_dtm, y_train[category])
    
    # calculating test accuracy
    prediction = RF_pipeline.predict(X_test_dtm)
    accuracy = accuracy + metrics.accuracy_score(y_test[category], prediction)
    print("Accuracy:{}".format(metrics.accuracy_score(y_test[category], prediction)))
    print("F1 score:{}".format(metrics.f1_score(y_test[category], prediction, average = 'macro')))
    print('AUC ROC is {}'.format(roc_auc_score(y_test[category],prediction)))
    print("\n")

print('Test averaged Accuracy is {}'.format(accuracy/len(category_df.columns[ :6])))

**Processing Action & Adventure titles...**
Accuracy:0.8443877551020408
F1 score:0.45781466113416325
AUC ROC is 0.5


**Processing Children & Family Movies titles...**
Accuracy:0.9030612244897959
F1 score:0.47453083109919575
AUC ROC is 0.5


**Processing Comedies titles...**
Accuracy:0.782312925170068
F1 score:0.4389312977099237
AUC ROC is 0.5


**Processing Documentaries titles...**
Accuracy:0.8622448979591837
F1 score:0.46301369863013697
AUC ROC is 0.5


**Processing Dramas titles...**
Accuracy:0.7389455782312925
F1 score:0.42493887530562346
AUC ROC is 0.5


**Processing International TV Shows titles...**
Accuracy:0.8690476190476191
F1 score:0.46496815286624205
AUC ROC is 0.5


Test averaged Accuracy is 0.8333333333333334
